In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "nmod"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-12-at-2023-06-13-04-38-304339d8.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_nmod_export_big_v1_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 72064,
 'annotations': [{'id': 22427,
   'completed_by': 1,
   'result': [{'value': {'start': 53,
      'end': 59,
      'text': 'Rootsi',
      'labels': ['nmod_phrases']},
     'id': 'VhCnnCifR1',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['free']},
     'id': 'y5BSa84KQl',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-06-09T14:35:28.305027Z',
   'updated_at': '2023-06-09T14:35:28.305060Z',
   'draft_created_at': None,
   'lead_time': 12.632,
   'prediction': {'id': 72063,
    'model_version': 'undefined',
    'created_ago': '31\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 53,
       'end': 59,
       'text': ['Rootsi'],
       'labels': ['nmod_phrases']}}],
    'score': None,
    

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,72064,Train/aja_ee199920_osa_3_ud211.conllu,Rootsis on Lindströmi nimetatud ka reeturiks :...,53,59,Rootsi,free,Rootsis on Lindströmi nimetatud ka reeturiks :...,100.0,100.0,100.0
1,72065,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,28,33,Saksa,free,"Nõuet kordasid avalikult ka diplomaadid , seal...",59.1,63.6,63.6
2,72066,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,59,70,välisasjade,bound,Nõuet kordasid avalikult ka Saksa diplomaadid ...,59.1,63.6,63.6
3,72067,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,130,133,oma,free,Nõuet kordasid avalikult ka Saksa diplomaadid ...,63.6,72.7,68.2
4,72068,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,140,151,Riigipäeval,free,Nõuet kordasid avalikult ka Saksa diplomaadid ...,63.6,68.2,68.2
...,...,...,...,...,...,...,...,...,...,...,...
2736,74926,Train/tea_toohoive.conllu,"On väidetud , et Eestis ei saa turumajanduse t...",31,44,turumajanduse,bound,"On väidetud , et Eestis ei saa tingimustes rää...",41.4,44.8,44.8
2737,74927,Train/tea_toohoive.conllu,"On väidetud , et Eestis ei saa turumajanduse t...",74,84,täiendõppe,bound,"On väidetud , et Eestis ei saa turumajanduse t...",41.4,44.8,44.8
2738,74928,Train/tea_toohoive.conllu,"On väidetud , et Eestis ei saa turumajanduse t...",125,151,täiend- ja ümberõppe järgi,bound,"On väidetud , et Eestis ei saa turumajanduse t...",50.0,53.8,53.8
2739,74929,Train/tea_toohoive.conllu,"On väidetud , et Eestis ei saa turumajanduse t...",181,192,elanikkonna,bound,"On väidetud , et Eestis ei saa turumajanduse t...",44.8,51.7,48.3


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,72064,Train/aja_ee199920_osa_3_ud211.conllu,Rootsis on Lindströmi nimetatud ka reeturiks :...,53,59,Rootsi,free,Rootsis on Lindströmi nimetatud ka reeturiks :...,100.0,100.0,100.0
1,72065,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,28,33,Saksa,free,"Nõuet kordasid avalikult ka diplomaadid , seal...",59.1,63.6,63.6
3,72067,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,130,133,oma,free,Nõuet kordasid avalikult ka Saksa diplomaadid ...,63.6,72.7,68.2
4,72068,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,140,151,Riigipäeval,free,Nõuet kordasid avalikult ka Saksa diplomaadid ...,63.6,68.2,68.2
5,72069,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,152,164,1897. aastal,free,Nõuet kordasid avalikult ka Saksa diplomaadid ...,66.7,71.4,71.4
...,...,...,...,...,...,...,...,...,...,...,...
2729,74894,Train/aja_ee200110_osa_10_ud211.conllu,Me räägime siin tervest traditsioonist ja teis...,79,83,tema,free,Me räägime siin tervest traditsioonist ja teis...,93.8,93.8,93.8
2730,74895,Train/aja_luup200202_osa_2_ud211.conllu,"Näib , et mitte ainult meedia , vaid ka tavali...",80,83,oma,free,"Näib , et mitte ainult meedia , vaid ka tavali...",92.9,100.0,92.9
2733,74898,Train/aja_horisont2000_osa_5_ud211.conllu,“ Inimese põlvnemises ” mõtisklebki Darwin nei...,120,123,oma,free,“ Inimese põlvnemises ” mõtisklebki Darwin nei...,94.9,94.9,100.0
2735,74925,Train/aja_ee200110_osa_10_ud211.conllu,"Nii , et viimane peab automaati oma kehaga kai...",32,35,oma,free,"Nii , et viimane peab automaati kehaga kaitsma...",92.9,92.9,92.9


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,72064,Train/aja_ee199920_osa_3_ud211.conllu,Rootsis on Lindströmi nimetatud ka reeturiks :...,53,59,Rootsi,free,Rootsis on Lindströmi nimetatud ka reeturiks :...,100.0,100.0,100.0
26,72090,Dev/aja_luup200009_ud211.conllu,“ Mul oli sõber Värskast .,16,24,Värskast,free,“ Mul oli sõber .,100.0,100.0,100.0
30,72094,Train/aja_ee200110_osa_3_ud211.conllu,"Ka kuuldub teatrimajast , et direktor Iljin on...",47,58,Monte-Carlo,free,"Ka kuuldub teatrimajast , et direktor Iljin on...",100.0,100.0,100.0
45,72109,Train/aja_ee199920_osa_1_ud211.conllu,"Mälli sõnul ei osanud Feer arvestada , et klim...",108,119,äriplaanile,free,"Mälli sõnul ei osanud Feer arvestada , et klim...",100.0,100.0,100.0
61,72125,Train/tea_dr8047.conllu,Võimalusi ω-3-rasvhapete sisalduse suurendamis...,50,71,munades ja linnulihas,free,Võimalusi ω-3-rasvhapete sisalduse suurendamis...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2698,74858,Train/aja_ml200247_osa_4_ud211.conllu,Eri hinnangutele tuginedes on Lacto võlg piima...,41,58,piimatootjate ees,free,Eri hinnangutele tuginedes on Lacto võlg üle 3...,100.0,100.0,100.0
2712,74877,Train/tea_geofyysika.conllu,"Osutus , et tuuma aine sulamistemperatuur tema...",42,46,tema,free,"Osutus , et tuuma aine sulamistemperatuur tege...",100.0,100.0,100.0
2713,74878,Train/tea_geofyysika.conllu,"Osutus , et tuuma aine sulamistemperatuur tema...",42,79,tema tegelikkuse ülikõrgetel rõhkudel,free,"Osutus , et tuuma aine sulamistemperatuur on m...",100.0,100.0,100.0
2714,74879,Train/tea_geofyysika.conllu,"Osutus , et tuuma aine sulamistemperatuur tema...",47,58,tegelikkuse,free,"Osutus , et tuuma aine sulamistemperatuur tema...",100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [10]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
2,72066,Train/aja_horisont2000_osa_1_ud211.conllu,Nõuet kordasid avalikult ka Saksa diplomaadid ...,59,70,välisasjade,bound,Nõuet kordasid avalikult ka Saksa diplomaadid ...,59.1,63.6,63.6
6,72070,Train/ilu_remsu_ud211.conllu,"Siiski on teada , et Marco Polo polnud sugugi ...",65,71,Hiinas,bound,"Siiski on teada , et Marco Polo polnud sugugi ...",92.9,92.9,96.4
7,72071,Train/ilu_remsu_ud211.conllu,"Siiski on teada , et Marco Polo polnud sugugi ...",146,179,nii Hiina kui ka kogu Aasia kohta,bound,"Siiski on teada , et Marco Polo polnud sugugi ...",90.9,90.9,100.0
8,72072,Train/aja_sloleht20071217_ud211.conllu,Eestimaa uhkustele kuulub tänu ning Ivo Lille ...,0,8,Eestimaa,bound,uhkustele kuulub tänu ning Ivo Lille loodud sü...,81.8,81.8,81.8
12,72076,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,Sõlmelised vormid kujutavad endast tüüpiliselt...,104,124,difuusse amüloidoosi,bound,Sõlmelised vormid kujutavad endast tüüpiliselt...,78.6,92.9,85.7
...,...,...,...,...,...,...,...,...,...,...,...
2734,74924,Dev/aja_luup200009_ud211.conllu,Tegelikult läks lõunast õhtul kella poole ühek...,24,29,õhtul,bound,Tegelikult läks lõunast kella poole üheksani .,71.4,71.4,71.4
2736,74926,Train/tea_toohoive.conllu,"On väidetud , et Eestis ei saa turumajanduse t...",31,44,turumajanduse,bound,"On väidetud , et Eestis ei saa tingimustes rää...",41.4,44.8,44.8
2737,74927,Train/tea_toohoive.conllu,"On väidetud , et Eestis ei saa turumajanduse t...",74,84,täiendõppe,bound,"On väidetud , et Eestis ei saa turumajanduse t...",41.4,44.8,44.8
2738,74928,Train/tea_toohoive.conllu,"On väidetud , et Eestis ei saa turumajanduse t...",125,151,täiend- ja ümberõppe järgi,bound,"On väidetud , et Eestis ei saa turumajanduse t...",50.0,53.8,53.8


In [11]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [12]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
251,72315,Train/aja_pm19980926e_ud211.conllu,"Näiteks võib joonistada pildi inimesest , keda...",30,58,"inimesest , keda enam ei ole",redundant comma,"Näiteks võib joonistada pildi , ning pildi vah...",100.0,100.0,100.0
378,72442,Train/aja_ee200110_osa_10_ud211.conllu,"Toomas Haug kirjutab Debora Vaarandist , kui X...",45,55,XX sajandi,redundant comma,"Toomas Haug kirjutab Debora Vaarandist , kui l...",100.0,100.0,100.0
633,72699,Train/tea_dr8047.conllu,Linaseemnete või linaseemnejahu söötmise mõju ...,79,111,"% ( Aymond , van Elswyk , 1995 )",redundant comma,Linaseemnete või linaseemnejahu söötmise mõju ...,100.0,100.0,100.0
1444,73513,Train/tea_geofyysika.conllu,Üks paremaid aegu astronoomia jaoks oli varase...,74,81,Kreekas,redundant comma,Üks paremaid aegu astronoomia jaoks oli varase...,58.8,76.5,64.7
1445,73514,Train/tea_geofyysika.conllu,Üks paremaid aegu astronoomia jaoks oli varase...,84,110,ajavahemikus 600-150 e. Kr,redundant comma,Üks paremaid aegu astronoomia jaoks oli varase...,78.6,92.9,78.6
1470,73539,Train/aja_luup200106_osa_2_ud211.conllu,"Tartus , 4. novembril 1956. aastal , Ungari ül...",22,34,1956. aastal,redundant comma,"Tartus , 4. novembril , Ungari ülestõusu toetu...",75.0,75.0,87.5
2461,74534,Train/tea_dr8047.conllu,"Broilerirasva rasvhappeline koostis , % üldlip...",38,76,"% üldlipiididest ( Ahn et al. , 1998 )",redundant comma,"Broilerirasva rasvhappeline koostis ,",100.0,100.0,100.0
2699,74859,Train/ilu_vilep_ud211.conllu,"Agnes Raag , neiupõlvenimega Kasemets , tihkus...",11,37,", neiupõlvenimega Kasemets",redundant comma,"Agnes Raag , tihkus juba nutta ja mees otsusta...",86.7,86.7,100.0


In [13]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [14]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
577,72643,Train/aja_ee200110_osa_3_ud211.conllu,""" Tean Eestit just briljantse ja kuulsa astron...",173,208,meid taevast ähvardavatest ohtudest,unnatural,""" Tean Eestit just briljantse ja kuulsa astron...",96.4,96.4,100.0
706,72772,Train/tea_eesti_arst_2004_osa_3_ud211.conllu,"Kokkuvõtteks võib ütelda , et nesiritiidi näol...",118,160,südamepuudulikkuse ägeda dekompensatsiooni,unnatural,"Kokkuvõtteks võib ütelda , et nesiritiidi näol...",90.9,90.9,90.9
883,72951,Train/aja_ee200110_osa_5_ud211.conllu,"Arstid , kes oma madala töötasu tõttu nii peen...",206,209,oma,unnatural,"Arstid , kes oma madala töötasu tõttu nii peen...",92.1,92.1,94.7
1371,73440,Train/aja_luup200202_osa_1_ud211.conllu,"Ei ole alust arvata , et viimase aasta jooksul...",47,72,nende kahe institutsiooni,unnatural,"Ei ole alust arvata , et viimase aasta jooksul...",73.3,73.3,86.7
1375,73444,Train/tea_eesti_arst_2004_osa_3_ud211.conllu,Perearstide “ dispetšerlust ” ehk patsiendi so...,135,138,oma,unnatural,Perearstide “ dispetšerlust ” ehk patsiendi so...,100.0,100.0,100.0
2355,74428,Train/aja_luup200202_osa_2_ud211.conllu,Nõnda esitab ta igaühele väljakutse tunda oma ...,42,45,oma,unnatural,Nõnda esitab ta igaühele väljakutse tunda indi...,100.0,100.0,100.0
2600,74673,Train/aja_pm19980926e_ud211.conllu,Kummutis on palju erinevaid sahtleid ning see ...,46,50,tema,unnatural,Kummutis on palju erinevaid sahtleid ning see ...,81.8,90.9,81.8
2711,74876,Train/tea_geofyysika.conllu,"Osutus , et tuuma aine sulamistemperatuur tema...",12,22,tuuma aine,unnatural,"Osutus , et sulamistemperatuur tema tegelikkus...",93.8,93.8,100.0


In [15]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [16]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [17]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1005,73074,Train/aja_pm20001004_osa_3_ud211.conllu,Guznetsovi ( teist vahialust - toim ) ta ei tu...,31,35,toim,other redundant punctuation,Guznetsovi ( teist vahialust - ) ta ei tundnud .,100.0,100.0,100.0


In [19]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

In [ ]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

df_un = df.loc[df['type'] == "unnatural"]
df_un

df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

df_dub = df.loc[df['type'] == "dubious"]
df_dub

df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:32:20.113247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:32:20.462991: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:32:21.649098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:32:21.649254: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data






920it [02:17,  6.69it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")